FRED Data Series with Fedapi

In [ ]:
pip install fredapi

In [ ]:
!pip install fredapi


In [ ]:
import pandas as pd
from fredapi import Fred


fred = Fred(api_key='f7c33df54eac626aedc8608eb50e8a96')

# Loading the data
file_path = 'data_dictionary_toFRED.xlsx'
data = pd.read_excel(file_path)

# Creating a new DataFrame with the required columns
new_data = pd.DataFrame(columns=["Series (from attached sheet)", "Series Name (from attached sheet)", "FRED Pnuemonic", "Level specification (from attached sheet)"])

# Populating the new DataFrame with the relevant columns from the original data
new_data["Series (from attached sheet)"] = data["Series"]
new_data["Series Name (from attached sheet)"] = data["Description"]
new_data["Level specification (from attached sheet)"] = data["Level Specification "]

# Function to search for series on FRED
def get_fred_series_id(series_name):
    try:
        search_results = fred.search(series_name)
        if not search_results.empty:
            # Return the first result's id
            return search_results.iloc[0]['id']
    except Exception as e:
        print(f"Error searching for {series_name}: {e}")
    return None

# Loop through each series and search for it on FRED
for i, row in new_data.iterrows():
    series_name = row["Series Name (from attached sheet)"]
    fred_id = get_fred_series_id(series_name)
    new_data.at[i, "FRED Pnuemonic"] = fred_id


new_data.to_excel('new_series_data.xlsx', index=False)

print("New series data saved successfully.")


Error searching for ADP Nonfarm Private Payroll Employment: Small businesses: 'NoneType' object has no attribute 'empty'
Error searching for ADP Nonfarm Private Payroll Employment: Medium businesses: 'NoneType' object has no attribute 'empty'
Error searching for ADP Nonfarm Private Payroll Employment: Large businesses: 'NoneType' object has no attribute 'empty'
Error searching for ADP Nonfarm Private Payroll Employment: Healthcare: 'NoneType' object has no attribute 'empty'
Error searching for Employment: Utilties: 'NoneType' object has no attribute 'empty'
Error searching for Aggregate Weekly Hours: Prod & Nonsupervisory, Private Industries: 'NoneType' object has no attribute 'empty'
Error searching for Average Weekly Hours: Prod & Nonsupervisory, Private Industries: 'NoneType' object has no attribute 'empty'
Error searching for Average Weekly Hours: Prod & Nonsupervisory, Goods-producing industries: 'NoneType' object has no attribute 'empty'
Error searching for Average Weekly Hours: 

FRED Data Series with Fedapi with Fuzzywuzzy Library

In [ ]:
pip install fuzzywuzzy python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from fredapi import Fred
from fuzzywuzzy import process, fuzz


fred = Fred(api_key='f7c33df54eac626aedc8608eb50e8a96')


file_path = 'data_dictionary_toFRED.xlsx'
data = pd.read_excel(file_path)


new_data = pd.DataFrame(columns=["Seriess", "Series Name", "FRED Pnuemonic", "Level specification"])


new_data["Seriess"] = data["Series"]
new_data["Series Name"] = data["Description"]
new_data["Level specification"] = data["Level Specification "]

# Function to search for series on FRED and use fuzzy matching for close matches
def get_fred_series_id(series_name):
    try:
        search_results = fred.search(series_name)
        if search_results is not None and not search_results.empty:
            best_match = process.extractOne(series_name, search_results['title'], scorer=fuzz.token_sort_ratio)
            if best_match:
                matched_title = best_match[0]
                matched_id = search_results[search_results['title'] == matched_title]['id'].values[0]
                return matched_id
    except Exception as e:
        print(f"Error searching for {series_name}: {e}")
    return None


for i, row in new_data.iterrows():
    series_name = row["Series Name"]
    fred_id = get_fred_series_id(series_name)
    new_data.at[i, "FRED Pnuemonic"] = fred_id


new_data.to_excel('new_series_data.xlsx', index=False)

print("New series data saved successfully.")


Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
New series data saved successfully.


Improved code with exact matches

In [ ]:
!pip install fuzzywuzzy


In [ ]:
import pandas as pd
from fredapi import Fred
from fuzzywuzzy import process, fuzz

# Replace 'your_api_key_here' with your actual FRED API key
fred = Fred(api_key='f7c33df54eac626aedc8608eb50e8a96')

# Load the data
file_path = 'data_dictionary_toFRED.xlsx'
data = pd.read_excel(file_path)

# Create a new DataFrame with the required columns
new_data = pd.DataFrame(columns=["Seriess", "Series Name", "FRED Pneumonic", "Level Specification"])

# Populate the new DataFrame with the relevant columns from the original data
new_data["Seriess"] = data["Series"]
new_data["Series Name"] = data["Description"]
new_data["Level Specification"] = data["Level Specification "]

# Function to search for series on FRED using both descriptions and titles for fuzzy matching
def get_fred_series_id(series_description, series_title):
    try:
        search_results = fred.search(series_description)
        # Check if search_results is None before proceeding
        if search_results is None or search_results.empty:
            return None  # Return None if no results found or if the result is None

        # If search_results is not None and not empty, proceed with fuzzy matching
        combined_search_results = search_results['title'].tolist() + [series_title]
        best_match = process.extractOne(series_description, combined_search_results, scorer=fuzz.token_sort_ratio)
        if best_match and best_match[1] >= 50:  # Lowering the threshold to 50 for broader matches
            matched_title = best_match[0]
            matched_id = search_results[search_results['title'] == matched_title]['id'].values[0] if matched_title in search_results['title'].tolist() else None
            return matched_id
    except Exception as e:
        print(f"Error searching for {series_description}: {e}")
    return None

# Loop through each series and search for it on FRED
for i, row in new_data.iterrows():
    description = row["Series Name"]
    title = row["Seriess"]  # Make sure to use the column name correctly
    fred_id = get_fred_series_id(description, title)
    new_data.at[i, "FRED Pneumonic"] = fred_id  # Correct spelling to match column header

# Save the new DataFrame to an Excel file
new_data.to_excel('improved_fred_data.xlsx', index=False)

print("Improved series data saved successfully.")


Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for nan: quote_from_bytes() expected bytes
Error searching for Inventories: Manufacturing: Too Many Requests.  Exceeded Rate Limit
Error searching for Inventories: Manufacturing Nondurable Goods: Too Many Requests.  Exceeded Rate Limit
Error searching for Inventories: Retail Trade: Too Many Requests.  Exceeded Rate Limit
Error searching for nan: quote_from_bytes() expected bytes
Error searching for Inventories: Manufacturing & Trade: Too Many Requests.  Exceeded Rate Limit
Error searching for Inventories: Merchant Wholesale Durable Goods: Too Many Requests.  Exceeded Rate Limit
Error searching for Inventories: Merchant Wholesale Trade: Too Many Requests.  Exceeded Rate Limit
Error searching for Inventories: Merchant Wholesale Nondurable G

Matlab code

In [ ]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime

# Define data series and other particulars
start_date = datetime(1975, 3, 31)
series_names = [
    'GS10',     # Treasury notes, 10-year
    'TB3MS',    # T-Bill Rate, 3-month yields
    'AAA',      # AAA Corp Bond Yield
    'BAA',      # BAA Corp Bond Yield
    'FEDFUNDS', # Fed Funds Rate
    'GDPC1',    # Real GDP
    'PCECC96',  # Real Personal Consumption Expenditures
    'PNFIC1',   # Real non-residential fixed investment
    'PRFIC1',   # Real residential fixed investment
    'SLCEC1',   # Real state and local government consumption/gross investment
    'CBIC1',    # Real change in inventories
    'EXPGSC1',  # Real exports of goods and services
    'IMPGSC1',  # Real imports of goods and services
    'CP',       # Corporate profits
    'INDPRO',   # Industrial production
    'HOUST',    # Housing starts
    'USSTHPI',  # FHFA U.S. total house price index
    'UNRATE',   # Unemployment rate
    'PAYEMS',   # Total payroll employment
    'GDPCTPI',  # GDP price index
    'CPIAUCSL', # Consumer price index
    'CPILFESL', # Core consumer price index
    'PCECTPI',  # PCE price index
    'PCEPILFE', # Core PCE price index
    'GDP'       # Nominal GDP
]

# Pre-allocate datatable
data_table = pd.DataFrame()

# Loop through each series and pull from FRED
for series in series_names:
    data = web.DataReader(series, 'fred', start_date)
    data = data.resample('Q').mean()  # Convert to quarterly frequency by taking mean
    data.columns = [series]          # Rename the column to the series name
    if data_table.empty:
        data_table = data
    else:
        data_table = data_table.join(data, how='outer')

# Output raw data to .csv file for archival purposes
data_table.to_csv('raw_data.csv')
